## How-to guide for Personalized Promotions use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build a model that creates personalized promotions using the Abacus.AI Python Client Library.

We'll be using the [Retail Interaction Logs](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/events.csv) and [Item Categories](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/item_categories.csv) datasets, which contain information about user interactions and item attributes.

1. Install the Abacus.AI library.

In [ ]:
!pip install abacusai

We'll also import pandas and pprint tools for visualization in this notebook.

In [ ]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [ ]:
#@title Abacus.AI API Key

api_key = '2fdecde877dc45fab937eff82b70eff0'  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [ ]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model catered specifically for your data.

We'll call the `list_use_cases` method to retrieve a list of the Use Cases currently available on the Abacus.AI platform.

In [ ]:
client.list_use_cases()

In this notebook, we're going to create a model that returns personalized promotions using the Retail Interaction Logs and Item Categories datasets. The 'PERS_PROMOTIONS' use case is best tailored for this situation.

In [ ]:
#@title Abacus.AI Use Case

use_case = 'PERS_PROMOTIONS'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [ ]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

Finally, let's create the project.

In [ ]:
promotions_project = client.create_project(name='Personalized Promotions Project', use_case=use_case)
promotions_project.to_dict()

**Note: When feature_groups_enabled is False then the use case does not support feature groups (collection of ML features). Therefore, Datasets are created at the organization level and tied to a project to further use them for training ML models**

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using two datasets for this notebook. We'll tell Abacus.AI how each dataset should be used when creating it by tagging the dataset with a special Abacus.AI **Dataset Type**.
- [Retail Interaction Logs](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/events.csv) (**USER_ITEM_INTERACTIONS**): 
This dataset contains information about visitor interactions, with data such as the visitor ID, event, timestamp, etc.
- [Item Categories](https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/item_categories.csv) (**CATALOG_ATTRIBUTES**): This dataset contains information about items with specified IDs.

### Add the datasets to Abacus.AI

First we'll use Pandas to preview the file, then add it to Abacus.AI.

In [ ]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/events.csv')

In [ ]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/pers_promotion/item_categories.csv')

Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [ ]:
user_item_dataset = client.create_dataset_from_file_connector(name='Retail Interaction Logs1',table_name='Retail_Interaction_Logs1',
                                     location='s3://realityengines.datasets/pers_promotion/events.csv',
                                     refresh_schedule='0 12 * * *')

item_attributes_dataset = client.create_dataset_from_file_connector(name='Item Categories1',table_name='Item_Categories1',
                                     location='s3://realityengines.exampledatasets/pers_promotion/item_categories.csv',
                                     refresh_schedule='0 12 * * *')

datasets = [user_item_dataset, item_attributes_dataset]

### Check dataset processing status and get dataset schemas when processing finishes

In [ ]:
for dataset in datasets:
    dataset.wait_for_inspection()
    print(f'{dataset.name} Schema:')
    pp.pprint(client.get_dataset_schema(dataset_id= dataset.dataset_id))

Each column in a **Dataset** is associated with a **Column Data Type**.
All we need to do now is to tell Abacus.AI what field we want to predict from our **Dataset**.

For each **Dataset Type** in a **Use Case**, there are special **Column Mappings** that can be applied to a column. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API.

In [ ]:
client.describe_use_case_requirements(use_case)[0].allowed_column_mappings

We would map the *itemid* column as **ITEM_ID**, the *visitorid* column as **USER_ID**, and the *timestamp* column as **TIMESTAMP**.

The result of this call will be the updated schema after the **Column Mapping** is applied.

In [ ]:
client.attach_dataset_to_project(dataset_id=user_item_dataset.dataset_id, project_id=promotions_project.project_id, dataset_type='USER_ITEM_INTERACTIONS')


client.set_column_mapping(project_id=promotions_project.project_id, dataset_id=user_item_dataset.dataset_id,
                          column='itemid', column_mapping='ITEM_ID')

client.set_column_mapping(project_id=promotions_project.project_id, dataset_id=user_item_dataset.dataset_id,
                          column='visitorid', column_mapping='USER_ID')

client.set_column_mapping(project_id=promotions_project.project_id, dataset_id=user_item_dataset.dataset_id,
                          column='timestamp', column_mapping='TIMESTAMP')

Now that we've uploaded the datasets, we're almost ready to train a model!

To be sure that our project is ready to go, let's call `project.validate` to confirm that all the project requirements have been met.

In [ ]:
promotions_project.validate()

## 3. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [ ]:
promotions_project.get_training_config_options()

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [ ]:
promotions_model = promotions_project.train_model(training_config={})
promotions_model.to_dict()

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated.

In [ ]:
promotions_model.wait_for_evaluation()

**Note that model training might take some minutes to some hours depending upon the size of datasets, complexity of the models being trained and a variety of other factors**

## **Checkpoint** [Optional]
As model training can take an hours to complete, your page could time out or you might end up hitting the refresh button, this section helps you restore your progress:

In [ ]:
!pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = ''  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
promotions_project = next(project for project in client.list_projects() if project.name == 'Personalized Promotions Project')
promotions_model = promotions_project.list_models()[-1]
promotions_model.wait_for_evaluation()

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics

In [ ]:
pp.pprint(promotions_model.get_metrics().to_dict())

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/PERS_PROMOTIONS/training) page.

## 4. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
promotions_deployment = client.create_deployment(name='Personalized Promotions Deployment',model_id=promotions_model.model_id,description='Personalized Promotions Deployment')
promotions_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = promotions_project.create_deployment_token().deployment_token
deployment_token

## 5. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `get_personalized_ranking` API call below.

This command will return a list with each specified item ID ranked according to the specified visitor's preferences. The ranking would be performed based on what movies the user liked in the past and how the movies and users are related to each other depending on their attributes.

In [ ]:
ApiClient().get_personalized_ranking(deployment_token=deployment_token,
                                     deployment_id=promotions_deployment.deployment_id,
                                     query_data={"visitorid":"100018","itemid":["424193","181204","385091","339763","344603"]})